In [2]:
from arcgis.gis import GIS
gis=GIS("home")

In [3]:
import arcpy
import requests
from zipfile import ZipFile
import io
import os

In [3]:
#download land use/land cover data from mn geocommons

land_use_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/tif_base_landcover_minnesota.zip"
land_use_raster = requests.get(land_use_url, allow_redirects=True)
open(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 path optimization\land_use_raster.zip", 'wb').write(land_use_raster.content)


97106583

In [4]:
#unzip and extract LU/LC files

with ZipFile(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 path optimization\land_use_raster.zip", 'r') as land_use_raster:
    land_use_raster.extractall(
        path=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 path optimization\land_use_raster_mn"
    )

In [5]:
#download DEM for Minnesota

dem_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip"
dem_raster = requests.get(dem_url, allow_redirects=True)
open(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 path optimization\dem_raster.zip", 'wb').write(dem_raster.content)


402844618

In [7]:
#unzip and extract DEM file

with ZipFile(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 path optimization\dem_raster.zip", 'r') as dem_raster:
    dem_raster.extractall(
        path=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 path optimization\dem_raster_mn"
    )

In [ ]:
#get slope classes from DEM

arcpy.ddd.Slope(
    in_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\dem_raster\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m",
    out_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Slope_digita2",
    output_measurement="DEGREE",
    z_factor=1,
    method="PLANAR",
    z_unit="METER",
    analysis_target_device="GPU_THEN_CPU"
)

In [8]:
#Clip Slope layer to just the area around the problem to save processing

arcpy.management.Clip(
    in_raster="Slope_digita1",
    rectangle="565687.665516329 4874825.41184745 581816.810050418 4888942.45638066",
    out_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Slope_digita1_Clip",
    in_template_dataset="clip_area",
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 2 Dory\\Default.gdb\\Slope_digita1_Clip'>

In [9]:
#Clip LU/LC layer to just the area around the problem to save processing

arcpy.management.Clip(
    in_raster="landcover_impervious_statewide2013_v2.tif",
    rectangle="565687.665516329 4874825.41184745 581816.810050418 4888942.45638066",
    out_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\land_use_Clip",
    in_template_dataset="clip_area",
    nodata_value="255",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 2 Dory\\Default.gdb\\land_use_Clip'>

In [10]:
#reclassify land use according to Dory's preferences

arcpy.ddd.Reclassify(
    in_raster="land_use_Clip",
    reclass_field="Value",
    remap="1 101 1; 101 104 3; 104 109 2; 109 111 4",
    out_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Reclass_land1",
    missing_values=0
)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 2 Dory\\Default.gdb\\Reclass_land1'>

In [11]:
#reclassify slope according to Dory's preferences


arcpy.ddd.Reclassify(
    in_raster="Slope_digita1",
    reclass_field="Value",
    remap="0 3 1; 3 6 2; 6 90 3",
    out_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Reclass_Slop1",
    missing_values="DATA"
)

<Result 'C:\\Users\\rockx057\\Documents\\ArcGIS\\Projects\\Lab 2 Dory\\Default.gdb\\Reclass_Slop1'>

In [18]:
#weighted toward land cover
land_weight=(arcpy.Raster("Reclass_Slop1") * .3)+(arcpy.Raster("Reclass_land1")*.7)
land_weight.save(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\dory_path2")

In [19]:
#weighted equally
equal_weight=(arcpy.Raster("Reclass_Slop1") * .5)+(arcpy.Raster("Reclass_land1")*.5)
equal_weight.save(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\dory_path3")

In [20]:
#weighted toward slope
slope_weight=(arcpy.Raster("Reclass_Slop1") * .7)+(arcpy.Raster("Reclass_land1")*.3)
slope_weight.save(r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\dory_path4")

In [21]:
#calculate optimal path with slope preferred
arcpy.sa.OptimalRegionConnections(
    in_regions="Dory_endpoints",
    out_feature_class=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Optimal_slope",
    in_barrier_data=None,
    in_cost_raster="slope_weight",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x000001DB35AFA540>

In [23]:
#calculate optimal path with both equally weighted
arcpy.sa.OptimalRegionConnections(
    in_regions="Dory_endpoints",
    out_feature_class=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Optimal_equal",
    in_barrier_data=None,
    in_cost_raster="equal_weight",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x000001DB35AFACF0>

In [22]:
#calculate optimal path with land cover preferred
arcpy.sa.OptimalRegionConnections(
    in_regions="Dory_endpoints",
    out_feature_class=r"C:\Users\rockx057\Documents\ArcGIS\Projects\Lab 2 Dory\Default.gdb\Optimal_land",
    in_barrier_data=None,
    in_cost_raster="land_weight",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x000001DB35AFA3F0>